In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Loading data using keras method

Split is 80/20 
80 percent training
20 percent validating

In [ ]:
training_dataset = tf.keras.utils.image_dataset_from_directory(
    './model_data/',
    validation_split=0.2,
    subset='training',
    seed=123,
    image_size=(300,300),
    batch_size=25
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    './model_data/',
    validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=(300,300),
    batch_size=25
)

# Get class names

Should be two: wet and dry

In [ ]:
categories = training_dataset.class_names
print(categories)

# Pre-process data

Normalizing rgb values by adding a rescaling layer

In [ ]:
norm_layer = layers.Rescaling(1./255)

# Optimize performance

Pre-fetching for fast training and validation

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = training_dataset.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

# Build CNN Model

2 2D Convolutional NN 

2 2D MaxPooling NN

2 classes

In [ ]:
num_classes = 2

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
fitting = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10
)

In [ ]:
model.summary()

In [ ]:
plt.plot(fitting.history['accuracy'], label='accuracy')
plt.plot(fitting.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
plt.plot(fitting.history['loss'], label='loss')
plt.plot(fitting.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 0.7])
plt.legend(loc='lower right')

In [ ]:
model.save('./models/version6/')